In [10]:
import dalex

In [1]:
import mlrun
project = mlrun.get_or_create_project(name='mlops', user_project=True, context='./')

> 2023-01-12 14:19:58,354 [info] loaded project mlops from MLRun DB


In [51]:
fn = mlrun.code_to_function(name='dalex', kind='job', handler='run_dalex',filename='data/MLOps22/project/src/dalex.py')

In [52]:
out = fn.run(local=True,
       params = {'df_train': project.get_artifact('outlier_removal_outlier_removal').to_dataitem(),
                 'target': 'MEDV'})

> 2023-01-12 15:19:52,082 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-01-12 15:19:52,108 [info] starting run dalex-run_dalex uid=7acfbef2599b4443b47c678f2ba5d227 DB=http://mlrun-api:8080
dataframe shape before dalex : (309, 14)
Preparation of a new explainer is initiated

  -> data              : 309 rows 13 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 309 values
  -> model_class       : xgboost.sklearn.XGBRegressor (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_default at 0x7f147627adc0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 5.01, mean = 22.8, max = 50.0
  -> model type        : regression will be used (default)
  -> residual

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
mlops-jovyan,...2ba5d227,0,Jan 12 15:19:52,completed,dalex-run_dalex,kind=owner=jovyanhost=mlrun-jupyter-5cd9c659c-2dpxf,,df_train=s3://mlrun/projects/mlops-jovyan/artifacts/2964ba8c39474f4785ca598db2365058/outlier_removal/0/outlier_removal.parquettarget=MEDV,,dalex_output


> 2023-01-12 15:20:06,281 [info] run executed, status=completed


In [ ]:
project.list_artifacts()

In [ ]:
import json
json.loads(project.get_artifact('dalex-run_dalex_dalex_output').to_dataitem().get())

In [74]:
params = {"model_class": "xgboost.XGBRegressor",
           "label_columns": "MEDV",
           "model_name": 'housing2',
           "random_state": 50}

for key,val in json.loads(project.get_artifact('dalex-run_dalex_dalex_output').to_dataitem().get()).items():
    params['TRAIN_' + key] = val
    
fn = mlrun.import_function('hub://auto_trainer')
fn.run(handler='train',
       inputs={"dataset": project.get_artifact('get_data_housing').target_path},
       params = params,
       local=True)

> 2023-01-12 15:27:45,168 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-01-12 15:27:45,190 [info] starting run auto-trainer-train uid=c98862a90f444b4594b03ed70fe4dada DB=http://mlrun-api:8080
> 2023-01-12 15:27:45,288 [info] test_set or train_test_split_size are not provided, setting train_test_split_size to 0.2
> 2023-01-12 15:27:45,323 [info] Sample set not given, using the whole training set as the sample set
> 2023-01-12 15:27:45,329 [info] training 'housing2'


> 2023-01-12 15:27:46,088 [info] run executed, status=completed


## French motor dataset

In [ ]:
import sklearn
from src.outlier_removal import *

outlier_removal = mlrun.code_to_function(name='outlier_removal', kind='job', image='mlrun/mlrun', handler='run', filename='src/outlier_removal.py')
outlier_removal.apply(mlrun.auto_mount())
if os.getenv('V3IO_ACCESS_KEY','False')=='False':
    outlier_removal.spec.disable_auto_mount=False
    
# Setting outlier removal params
votes_thresholds = 3
pyod_contamination=0.2 # + (0,0.5)
z_score_threshold=3
iqr_low=0.01
iqr_high=0.99
iqr_max_removal_percent_per_column=0.95
remove_outliers_functions = [(remove_outliers_z_score, {'threshold': z_score_threshold}),
                    (remove_outliers_iqr, {'low_quantile': iqr_low, 'high_quantile':iqr_high, 'max_removal_percent_per_column':iqr_max_removal_percent_per_column}),
                    (remove_outliers_LOF, {'contamination': pyod_contamination}),
                    (remove_outliers_ABOD, {'contamination': pyod_contamination}),
                       (remove_outliers_HBOS, {'contamination': pyod_contamination})
                        ]

outlier_removal_run = outlier_removal.run(inputs={'dataitem': get_data_run.outputs[dataset]},
                                          params={'remove_outliers_functions': remove_outliers_functions, 
                                                'remove_outlier': remove_outlier,
                                                'votes_thresholds': votes_thresholds,
                                                'label_column': label_column,
                                                'random_state': 50},
                                          outputs=['outlier_removal', 'outlier_removal_test'])

In [ ]:
fn.run(local=True,
       params = {'df_train': project.get_artifact('outlier_removal_outlier_removal').to_dataitem(),
                 'df_test' : project.get_artifact('outlier_removal_outlier_removal_test').to_dataitem(),
                 'target': 'MEDV'})